## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

In [10]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import decomposition
import seaborn

# Import code
from brats.load_data import load_data
from sklearn.model_selection import train_test_split

# scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Classifiers
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm

#### Split data
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 55% and a test set of 45% of the subjects.
- Bepalen wat test_size wordt. 
- output bij train_test split functie



In [18]:
data_features = pd.DataFrame(data=data_brats)
data_train, data_test = train_test_split(data_features, test_size=0.45) # Nog bepalen wat test_size wordt
#print(f'data_train: {data_train}')
#print(f'data_test: {data_test}')

#### No None
Deleting columns with NaN or filling them.
- Bepalen waar threshold ligt

In [20]:
# Inzicht in data
print(f'OVERZICHT: {data_train.isnull().sum()}')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*data_train.shape[0] + 1)
data_dropcolumn = data_train.dropna(axis=1, thresh=min_count)
#print(data_dropcolumn)
#print(data_dropcolumn.size)

# fill the NaN observations.
data_fill = data_dropcolumn.fillna(data_dropcolumn.median()) #Bekijken mean of median
#print(data_fill)
#print(data_fill.size)

# Inzicht in data
print(f'OVERZICHT NONONE: {data_fill.isnull().sum()}')

OVERZICHT: VOLUME_ET       0
VOLUME_NET      0
VOLUME_ED       0
VOLUME_TC       0
VOLUME_WT       0
               ..
TGM_Cog_X_6    91
TGM_Cog_Y_6    91
TGM_Cog_Z_6    91
TGM_T_6        91
label           0
Length: 725, dtype: int64


C:\Users\Lishia\AppData\Local\Temp\ipykernel_24508\1875792392.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_fill = data_dropcolumn.fillna(data_dropcolumn.median()) #Bekijken mean of median


OVERZICHT NONONE: VOLUME_ET      0
VOLUME_NET     0
VOLUME_ED      0
VOLUME_TC      0
VOLUME_WT      0
              ..
TGM_Cog_X_1    0
TGM_Cog_Y_1    0
TGM_Cog_Z_1    0
TGM_T_1        0
label          0
Length: 705, dtype: int64


#### Split data in X and y
Split in X (data) and y (label)
- Checken of splitten van data en labels inderdaad moet na het er uithalen van NaN's 

In [22]:
y = data_fill.pop('label')
X = data_fill

#### Scale features
- Checken of Robust Scaler juiste keuze is

In [23]:
# standard scaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled_one = scaler.transform(X)
print(X_scaled_one)

# minmax scaler
scaler_two = MinMaxScaler()
scaler_two.fit(X)
X_scaled_two = scaler_two.transform(X)
print(X_scaled_two)

# robustscaler
scaler_three = RobustScaler()
scaler_three.fit(X)
X_scaled = scaler_three.transform(X)
print(X_scaled)

[[-1.02947975 -0.39842764 -0.82184863 ... -1.91960116 -0.76537983
  -1.11753984]
 [-1.00084585  1.59756712 -1.01725672 ... -0.03183738  0.81315031
   0.67351092]
 [-0.47894876 -0.72296177  1.01502196 ... -1.93942162 -0.07950401
  -0.79298959]
 ...
 [ 1.48695824 -0.59660762  1.59513821 ...  1.50231241 -0.30583326
   0.14916238]
 [ 2.74335277 -0.05245009 -0.06716059 ...  0.76444149  0.73233359
   1.19772214]
 [ 0.39527195 -0.29928273 -0.33423967 ...  1.77656263  0.3226358
   0.22285711]]
[[0.00118861 0.07596838 0.16121198 ... 0.00483228 0.2350806  0.03067322]
 [0.00833291 0.46039502 0.11608423 ... 0.46507421 0.59978766 0.37616523]
 [0.13854888 0.01346342 0.58542079 ... 0.         0.39354685 0.09327866]
 ...
 [0.62905265 0.03779911 0.71939344 ... 0.83910409 0.34125525 0.27501889]
 [0.94252946 0.14260332 0.33550042 ... 0.65920916 0.58111559 0.47728509]
 [0.35667139 0.09506359 0.2738209  ... 0.90596704 0.48645812 0.28923454]]
[[-0.53644033 -0.05428053 -0.4690925  ... -1.20910833 -0.42434121

#### Transform features
- We denken alleen PCA te gebruiken. Klopt het dat je dan niet ook selectie gebruikt?
- PCA gaat uit van lineair model. Hoe kunnen we weten of ons onze data daar geschikt voor is?
- Is het de bedoeling dat we ons hier verder in verdiepen of valt dat buiten de scope van het vak?
- Uitzoeken hoe we de X_test en y_test correct gereed krijgen voor PCA.

In [25]:
# Perform a PCA
pca = decomposition.PCA(n_components=2)
pca.fit(X_scaled) 
X_train_pca = pca.transform(X_scaled)

'''
X_test_pca = pca.transform(X_test)

# Fit kNN
knn = neighbors.KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_pca, y_train)
score_train = knn.score(X_train_pca, y_train)
score_test = knn.score(X_test_pca, y_test)

# Print result
print(f"Training result: {score_train}")
print(f"Test result: {score_test}")
'''

'\nX_test_pca = pca.transform(X_test)\n\n# Fit kNN\nknn = neighbors.KNeighborsClassifier(n_neighbors=15)\nknn.fit(X_train_pca, y_train)\nscore_train = knn.score(X_train_pca, y_train)\nscore_test = knn.score(X_test_pca, y_test)\n\n# Print result\nprint(f"Training result: {score_train}")\nprint(f"Test result: {score_test}")\n'